### First Attempt

In [ ]:
!pip3 install -U sentence-transformers

In [ ]:
!pip3 install qdrant_client

In [ ]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer
import pandas as pd

In [ ]:
encoder = SentenceTransformer("all-MiniLM-L6-v2")

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/chaabi_data/bigBasketProducts.csv')

In [ ]:
df.head()

,index,product,category,sub_category,brand,sale_price,market_price,type,rating,description
0,1,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.0,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...
1,2,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.0,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ..."
2,3,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.0,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m..."
3,4,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.0,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...
4,5,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.0,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...


In [ ]:
#convert the dataframe df to a list of dictionaries but since the description can be multiline, store the description as a multiline string uisng ''' '''
documents = []
for index, row in df.iterrows():
    documents.append({
        'product': row['product'],
        'category': row['category'],
        'sub category': row['sub_category'],
        'brand': row['brand'],
        'sale price': row['sale_price'],
        'market price': row['market_price'],
        'type': row['type'],
        'rating': row['rating'],
        'description': '''{}'''.format(row['description'])
    })

# data = df.to_dict('records')

In [ ]:
qdrant = QdrantClient(":memory:")

In [ ]:
qdrant.recreate_collection(
    collection_name="big basket data",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),  # Vector size is defined by used model
        distance=models.Distance.COSINE,
    ),
)

True

In [ ]:
qdrant.upload_records(
    collection_name="big basket data",
    records=[
        models.Record(
            id=idx,
            vector=encoder.encode(doc["description"]),
            payload=doc
        )
        for idx, doc in enumerate(documents)
    ],
)

ValidationError: ignored

In [ ]:
hits = qdrant.search(
    collection_name="big basket data",
    query_vector=encoder.encode("anti dandruff scalp conditioning shampoo").tolist(),
    query_filter=models.Filter(
        must=[models.FieldCondition(key="sale price", range=models.Range(lte=1000))]
    ),
    limit=5,
)
for hit in hits:
    print(hit.payload, "score:", hit.score)

{'product': 'Anti-Dandruff Shampoo', 'category': 'Beauty & Hygiene', 'sub category': 'Hair Care', 'brand': 'Kaya Clinic', 'sale price': 212.5, 'market price': 250.0, 'type': 'Shampoo & Conditioner', 'rating': 4.4, 'description': 'The anti-dandruff shampoo is fortified with piroctone olamine that effectively removes dandruff. It contains Vitamin B5 and seaweed extract enriched scalp purifying factor that nourishes, protects the scalp against flaking and prevents it against dryness, thereby increasing its inherent health.'} score: 0.8426064073253744
{'product': 'Dan-O-Pause Anti Dandruff Shampoo', 'category': 'Beauty & Hygiene', 'sub category': 'Hair Care', 'brand': 'SSCPL HERBALS', 'sale price': 210.0, 'market price': 210.0, 'type': 'Shampoo & Conditioner', 'rating': 3.0, 'description': 'Dan-O-Pause Anti Dandruff Shampoo provides relief on sensitive scalps. It helps fight against persistent and recurrent dandruff conditions.'} score: 0.8243784944479512
{'product': 'Dan-O-Pause Anti Dand

###Second Attempt

In [ ]:
!pip3 install transformers fastapi uvicorn qdrant-client

In [ ]:
from qdrant_client import QdrantClient

# Replace these with your Qdrant instance details
qdrant_host = "http://your-qdrant-instance-url.com"
collection_name = "products"

# Initialize Qdrant client
client = QdrantClient(qdrant_host)

# Assuming you have embeddings and ids to upload
ids = [1, 2, 3, 4, 5]  # Replace with your IDs
embeddings = [
    [0.1, 0.2, 0.3, ...],  # Replace with your actual embeddings
    [0.2, 0.3, 0.4, ...],
    ...
]
# Index the embeddings to Qdrant
client.insert(collection_name=collection_name, data={"ids": ids, "vectors": embeddings})


AttributeError: ignored

In [ ]:
!curl \
    -X GET 'https://f5fec074-703f-4e90-9130-539be114010e.us-east4-0.gcp.cloud.qdrant.io:6333' \
    --header 'api-key: aUUf7XmMa8A1zuzTp_gUOw5pG9QnAAiHVytCKqVY6ER2yAmclRV7ug'

{"title":"qdrant - vector search engine","version":"1.6.1"}